<a href="https://colab.research.google.com/github/genaiconference/Agentic_RAG_Workshop/blob/main/04_Traditional_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Traditional RAG

In [1]:
!git clone https://github.com/genaiconference/Agentic_RAG_Workshop.git

Cloning into 'Agentic_RAG_Workshop'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 71 (delta 28), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (71/71), 2.24 MiB | 3.18 MiB/s, done.
Resolving deltas: 100% (28/28), done.


## Setup and Installations
Install necessary libraries for document processing, data handling, and interacting with Azure Document Intelligence and OpenAI.

In [1]:
!pip install -r /content/Agentic_RAG_Workshop/requirements.txt

  Using cached azure_ai_documentintelligence-1.0.2-py3-none-any.whl.metadata (53 kB)
  Using cached pathlib-1.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached pymupdf-1.26.3-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (3.4 kB)
  Using cached tools-1.0.2-py3-none-any.whl.metadata (1.4 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached langchain_openai-0.3.28-py3-none-any.whl.metadata (2.3 kB)
  Using cached rapidfuzz-3.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached chromadb-1.0.16-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies

## Load Environment Variables and Initialize Clients
Load environment variables containing API keys and endpoint information, and initialize the Azure Document Intelligence and OpenAI clients.

In [1]:
import os

os.chdir("/content/Agentic_RAG_Workshop/")

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

load_dotenv()

llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4.1",
    temperature=0,
)

embeddings = OpenAIEmbeddings(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="text-embedding-3-small"
)

## Load Parent Docs

In [2]:
import pickle

with open("parent_docs_and_ids.pkl", "rb") as f:
    parent_data = pickle.load(f)

parent_docs = parent_data["parent_docs"]
doc_ids = parent_data["doc_ids"]

## Load Child docs

In [3]:
from multivector_utils import create_child_documents, generate_summaries, generate_hypothetical_questions

id_key = "doc_id"

child_docs = create_child_documents(parent_docs, doc_ids, id_key)
summaries = generate_summaries(parent_docs, llm, id_key, doc_ids)
questions = generate_hypothetical_questions(parent_docs, id_key, doc_ids)

## Define VectorStore

**Azure AI Search**

In [11]:
from langchain_community.vectorstores.azuresearch import AzureSearch

index_name: str = "agentic-rag-workshop"
search_type: str = "semantic_hybrid"

vectorstore: AzureSearch = AzureSearch(azure_search_endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"),
                                            azure_search_key=os.getenv("AZURE_SEARCH_KEY"),
                                            index_name=index_name,
                                            search_type=search_type,
                                            semantic_configuration_name="agentic-rag-semantic-config",
                                            embedding_function=embeddings.embed_query)

**Chroma**

In [4]:
from langchain.vectorstores import Chroma

vectorstore = Chroma(
    collection_name="full_documents", embedding_function=embeddings
)

# Add child_docs, sumamries, questions to the vectorstore
vectorstore.add_documents(documents=child_docs)
vectorstore.add_documents(documents=summaries)
vectorstore.add_documents(documents=questions)

/tmp/ipython-input-383506511.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


['1ec7aed4-f414-4a39-a606-79ff30045f95',
 '7a3f5746-94d0-4291-94d9-dbd6be3b03b1',
 '372792d4-c5cf-4ef9-b99c-86e7e48b427b',
 '0b238e73-15f4-43bb-9281-0af337aca364',
 '2f1e611d-01cf-4510-9fb6-9c7299c9068d',
 '5edf852f-59ff-45a7-9376-2a6a96b85a72',
 'e150870b-88df-40e8-b644-dfa663852754',
 '96df5ac3-36aa-4510-8870-0e632e281901',
 'fc8cc7f3-b033-41f8-834c-9d312ea9b32d',
 'a78a4ce1-9f5f-464b-b6de-1b21941f349e',
 'ac79bcac-0a55-4b9c-8ff7-5ce425551a23',
 '7a8350d5-4ff6-4496-a537-c26b597059f7',
 'd5c3984d-96ce-4e7c-a735-b01ba901df91',
 '7998413f-0e2d-4994-8850-0c262862f95e',
 'ceef0a7e-fe58-40fb-948c-430ec0e4ad55',
 'ecb2871a-6886-4c6d-ba5d-ad7bbbef0287',
 'ede8f6bb-2db7-4428-9b29-72ce41e9d451',
 'b25322b8-42ab-4579-b686-f1e9d719d915',
 '103426f9-4fa1-4cf2-8ec9-d2cd54956f1c',
 '08faf9b4-2cb0-46fb-81b6-27b1abf83973',
 'c90687d3-5a32-44c5-9d17-7587256f7e99',
 '8a7d7a15-a655-4b79-b474-43c2379f8f92',
 'b3256393-42a5-48ff-8f3b-9e65d538c248',
 '9069548f-d3a8-41c8-b9fc-b19edc1bf98b',
 '56a695c5-66e2-

In [5]:
vectorstore.similarity_search("maternity_benefits")

[Document(metadata={'source_type': 'summary', 'doc_id': '25'}, page_content='**Summary of Maternity Benefits:**\n\n- **Eligibility:** Maternity benefits are available only for in-patient hospitalizations in India. Employees with more than two living children are not eligible. Expenses for voluntary medical termination within the first 12 weeks, infertility treatment, and sterilization are not covered.\n- **Coverage Details:**\n  - Maximum benefit: INR 75,000 for normal delivery; INR 1,00,000 for C-section (within the sum insured limit).\n  - 9-month waiting period is waived.\n  - Pre- and post-natal expenses are covered up to INR 5,000, in addition to the maternity limit.\n  - Newborns are covered from day 1, provided HR is informed within 15 days of birth.\n- **Important:** Claims must be submitted within 30 days; inform HR immediately about newborn coverage to ensure eligibility and avoid claim denial.'),
 Document(metadata={'source_type': 'Children', 'parent_id': '25', 'Header_2': '

## Create MultiVectorRetriver

In [7]:
from multivector_utils import create_MVR

retriever = create_MVR(parent_docs, doc_ids, vectorstore)
retriever.invoke("maternity_benefits")

[Document(metadata={'Header_1': 'Employee Benefits Manual 2023-24 Novartis Group', 'Header_2': 'Your Plan Details Maternity Benefits', 'page_number': 11, 'custom_metadata': 'Your Plan Details Maternity Benefits', 'source': 'Insurance Policy'}, page_content='##Your Plan Details Maternity Benefits\n\n · Maternity benefits are admissible only if the expenses are incurred in Hospital / Nursing Home as in-patients in India.  \n· Those Insured Persons who already have two or more living children will not be eligible for this benefit.  \n· Expenses incurred in connection with voluntary medical termination of pregnancy during the first 12 weeks from the date\nof conception are not covered. Infertility Treatment and sterilization are excluded from the policy.  \n### The maternity benefit under your Group Medical Plan  \n<table>\n<tr>\n<td>Maximum Benefit</td>\n<td>INR 75,000 for Normal &amp; INR 1 Lac (New) for C - Section within Sum Insured Limit</td>\n</tr>\n<tr>\n<td>9-months waiting period<

## Traditional RAG

### Define Prompt

In [8]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Avoid using generic phrases like "Provide context" or "as per context.

Question: {input}

Context: {context}

Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

### Create Simple RAG Chain

In [9]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Create a chain to combine documents
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

# Create a retrieval chain
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

### Test Q&A

In [10]:
from IPython.display import Markdown

question = "I am planning to adopt a kid. How many leaves can I avail and What are the Insurance benefits I can get?"
response = rag_chain.invoke({"input": question})

Markdown(response["answer"])

You are eligible for 26 weeks of parental leave following the adoption of a child, which must be availed within one year of the adoption. For insurance benefits, you can enroll your adopted child as a dependent within 30 days of adoption, making them eligible for coverage under your group medical plan from day 1. The plan covers hospitalization, pre-existing diseases, and day care procedures, among other benefits.